# HEN Autosolver -- WIP, I may add auto-pairing

In [ ]:
# @title Default title text
import pandas as pd
import numpy as np
import matplotlib as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10)

stream_count = 0

# approach temp
while True:
    min_approach_temp = input("Please enter the Minimum Approach Temperature [deg C]\n")
    if min_approach_temp.isdigit():
        min_approach_temp = float(min_approach_temp)
        break
    else:
        print("\nPlease enter a SINGLE number!!!!!!! AHHHHHH\n")

# temp src
while True:
    error_check = False

    stream_temp_src_str = input("Enter the list of Stream SOURCE Temperatures separated by SPACES [deg C]\n")
    stream_temp_src_str = stream_temp_src_str.split()

    stream_temp_src = []

    for temp_str in stream_temp_src_str:
        try:
            temp_value = float(temp_str)
            stream_temp_src.append(temp_value)
        except ValueError:
            print(f"\nInvalid input: {temp_str} is not a valid numeric input!!\n")
            error_check = True
            break

    print(f"\nYou entered a following list of Temperature IN: {stream_temp_src}")

    if error_check:
        continue
    else:
        stream_count = len(stream_temp_src)
        print(f"\nYou have entered {stream_count} stream(s).\n")
        break

# Temp tgt
while True:
    error_check = False

    stream_temp_tgt_str = input("Enter the list of Stream TARGET Temperatures separated by SPACES [deg C]\n")
    stream_temp_tgt_str = stream_temp_tgt_str.split()

    stream_temp_tgt = []

    for temp_str in stream_temp_tgt_str:
        try:
            temp_value = float(temp_str)
            stream_temp_tgt.append(temp_value)
        except ValueError:
            print(f"\nInvalid input: {temp_str} is not a valid numeric input!!")
            error_check = True
            break

    print(f"\nYou entered a following list of Temperature OUTPUT: {stream_temp_tgt}")

    if error_check:
        continue
    elif len(stream_temp_tgt) != stream_count:
        print("Stream counts don't match!!")
        print(f"\nYou have entered a list with {len(stream_temp_tgt)} streams, when the input had {stream_count} streams")
        continue
    else:
        break

# CP data
while True:
    error_check = False

    stream_cp_str = input("\nEnter the list of CP separated by SPACES [kW/C]\n")
    stream_cp_str = stream_cp_str.split()

    stream_cp = []

    for cp_str in stream_cp_str:
        try:
            cp_value = float(cp_str)
            stream_cp.append(cp_value)
        except ValueError:
            print(f"\nInvalid input: {cp_str} is not a valid numeric input!!")
            error_check = True
            break

    print(f"\nYou entered a following list of CP: {stream_cp}")

    if error_check:
        continue
    elif len(stream_cp) != stream_count:
        print("\nStream counts don't match!!")
        print(
            f"\nYou have entered a list with {len(stream_cp)} streams, when the input had {stream_count} streams")
        continue
    else:
        break

stream_df = pd.DataFrame({'t_src': stream_temp_src, 't_tgt': stream_temp_tgt, 'cp': stream_cp})

def CalcSourceIntervalTemp(temp_in, temp_out):
    if temp_in > temp_out:
        return temp_in - 0.5 * min_approach_temp
    else:
        return temp_in + 0.5 * min_approach_temp

def CalcTargetIntervalTemp(temp_in, temp_out):
    if temp_in > temp_out:
        return temp_out - 0.5 * min_approach_temp
    else:
        return temp_out + 0.5 * min_approach_temp

def CalcHeatLoad(temp_in, temp_out, cp_value):
    return abs((temp_in - temp_out) * cp_value)

def CalcCpSum():
    bitmask_matrix = []

    def CalcBitMask():
        for index, row in stream_df.iterrows():
            interval_min = row[["src_interval_temp", "tgt_interval_temp"]].min()
            interval_max = row[["src_interval_temp", "tgt_interval_temp"]].max()

            greaterthanbitmask = stream_intervals_df['unique_temp_intervals'] < interval_max
            betweenbitmask = stream_intervals_df['unique_temp_intervals'].between(interval_min, interval_max, inclusive='both')

            bitmask = greaterthanbitmask & betweenbitmask
            bitmask_matrix.append(bitmask.tolist())

        return bitmask_matrix

    stream_cp_sum_df = pd.DataFrame(CalcBitMask())

    for index, row in stream_cp_sum_df.iterrows():
        multiplier = -1 if stream_df['is_cooled_stream'][index] else 1
        multiplied_row = row*multiplier
        stream_cp_sum_df.iloc[index]=multiplied_row

    #print(stream_cp_sum_df)
    stream_cp_sum_df = stream_cp_sum_df.mul(stream_df['cp'], axis=0)
    return stream_cp_sum_df.sum()

def CalCumSumH():
    cumsum_h = []
    cum_h = -stream_intervals_df['cascade_h'].min()

    for index, row in stream_intervals_df.iterrows():
        cum_h -= row['delta_h']
        cumsum_h.append(cum_h)

    return cumsum_h

stream_df["src_interval_temp"] = stream_df.apply(lambda row: CalcSourceIntervalTemp(row['t_src'], row['t_tgt']), axis=1)
stream_df["tgt_interval_temp"] = stream_df.apply(lambda row: CalcTargetIntervalTemp(row['t_src'], row['t_tgt']), axis=1)
stream_df["heat_load"] = stream_df.apply(lambda row: CalcHeatLoad(row['t_src'], row['t_tgt'], row['cp']), axis=1)
stream_df["is_cooled_stream"] = stream_df['t_src'] > stream_df['t_tgt']
stream_df["max_interval_temp"] = stream_df.apply(lambda row: max(row['src_interval_temp'], row['tgt_interval_temp']), axis=1)
stream_df["min_interval_temp"] = stream_df.apply(lambda row: min(row['src_interval_temp'], row['tgt_interval_temp']), axis=1)

stream_intervals_df = pd.DataFrame({'unique_temp_intervals': sorted(
    pd.concat([stream_df["src_interval_temp"], stream_df["tgt_interval_temp"]]).unique(), reverse=True)})
stream_intervals_df = stream_intervals_df.sort_values(by='unique_temp_intervals', ascending=False).reset_index(drop=True)
stream_intervals_df['interval_delta_temp'] = (-stream_intervals_df['unique_temp_intervals'].diff()).fillna(0)
stream_intervals_df['cp_sum']=CalcCpSum()
stream_intervals_df['delta_h']=stream_intervals_df['cp_sum']*stream_intervals_df['interval_delta_temp']
stream_intervals_df['cascade_h']=(-stream_intervals_df['delta_h']).cumsum()
stream_intervals_df['cumsum_h'] = CalCumSumH()

unique_interval_temps_count = len(stream_intervals_df)

pinch_temp = stream_intervals_df.loc[stream_intervals_df['cumsum_h'] == 0, 'unique_temp_intervals'].iloc[0]
heating_util_required = stream_intervals_df['cumsum_h'].iloc[0]
cooling_util_required = stream_intervals_df['cumsum_h'].iloc[-1]

print("--------------------------------------------------------------------------------")
print(f"\nPinch Temperature is:  [{pinch_temp}] ")
print(f"Heating Utility Required is:  [{heating_util_required}] ")
print(f"Cooling Utility Required is:  [{cooling_util_required}] ")


Please enter the Minimum Approach Temperature [deg C]
20
Enter the list of Stream SOURCE Temperatures separated by SPACES [deg C]
245 200 30 90

You entered a following list of Temperature IN: [245.0, 200.0, 30.0, 90.0]

You have entered 4 stream(s).

Enter the list of Stream TARGET Temperatures separated by SPACES [deg C]
80 50 200 200

You entered a following list of Temperature OUTPUT: [80.0, 50.0, 200.0, 200.0]

Enter the list of CP separated by SPACES [kW/C]
3 4 5 3

You entered a following list of CP: [3.0, 4.0, 5.0, 3.0]
--------------------------------------------------------------------------------

Pinch Temperature is:  [100.0] 
Heating Utility Required is:  [115.0] 
Cooling Utility Required is:  [30.0] 


In [ ]:
print(stream_intervals_df)
print(stream_df)


   unique_temp_intervals  interval_delta_temp cp_sum delta_h cascade_h  \
0      235.0                    0.0              0.0     0.0      -0.0   
1      210.0                   25.0             -3.0   -75.0      75.0   
2      190.0                   20.0              5.0   100.0     -25.0   
3      100.0                   90.0              1.0    90.0    -115.0   
4       70.0                   30.0             -2.0   -60.0     -55.0   
5       40.0                   30.0              1.0    30.0     -85.0   

   cumsum_h  
0     115.0  
1     190.0  
2      90.0  
3       0.0  
4      60.0  
5      30.0  
   t_src  t_tgt   cp  src_interval_temp  tgt_interval_temp  heat_load  \
0  245.0   80.0  3.0      235.0               70.0              495.0   
1  200.0   50.0  4.0      190.0               40.0              600.0   
2   30.0  200.0  5.0       40.0              210.0              850.0   
3   90.0  200.0  3.0      100.0              210.0              330.0   

   is_cooled_stre